# [MATH2504 Programming of Simulation, Analysis, and Learning Systems at The University of Queensland](https://courses.smp.uq.edu.au/MATH2504/)

## Semester 2, 2024

# Practical E: Towards project 1

Note that we use the [GitHub repo](https://github.com/yoninazarathy/2504_2024_project1) for the base [project](https://courses.smp.uq.edu.au/MATH2504/assessment_html/project1.html).

In [ ]:
using Pkg;
# To be able to run this, have the worksheet inside the project repository, it should not be part of your assignment submission. 
Pkg.activate(".");
# Pkg.instantiate(); #This should be uncommented when the project is first run
include("poly_factorization_project.jl");

# We'll create a rational function type which is a ratio of two polynomials.

$$
r(x) = \frac{p(x)}{q(x)}.
$$

Ideally such a function would allow a representation where any joint factors are cancelled out. However we won't get to this. 

The purpose of creating such a type is to get a feel for what it is like to create another type that uses the `Polynomial` type.

In [ ]:
struct RationalFunction
    numerator::Polynomial
    denominator::Polynomial
end

To create `x` we specify the type `PolynomialDense`. To find where this type is defined press command/control+t, and type PolynomialDense. This can be done with any object defined in the project.

In [ ]:
x = x_poly(PolynomialDense)
r1 = RationalFunction(5x^2-3x+4, 6x^4-2x+5)
r2 = RationalFunction(-3x+4, 2x^2-2x+5)

@show r1
@show r2;

# We can create a `show` method

In [ ]:
import Base: show

The IO argument in `show` ensures that this function is called whenever a `RationalFunction` object is displayed.

In [ ]:
function show(io::IO, r::RationalFunction) 
    println(io, r.numerator)
    num_chars = max(length(string(r.numerator)),length(string(r.denominator)))
    println(io,"-"^num_chars)
    println(io,r.denominator)
end

In [ ]:
r1

In [ ]:
r2

# We can allow multiplication

In [ ]:
import Base: *

In [ ]:
*(rf1::RationalFunction, rf2::RationalFunction) =
         RationalFunction(rf1.numerator * rf2.numerator, rf1.denominator * rf2.denominator)

In [ ]:
r1*r2

# We can create a derivative function

In [ ]:
function derivative(r::RationalFunction)
    n = r.numerator
    d = r.denominator
    
    #The quoutient rule for derivative
    RationalFunction(derivative(n)*d - n*derivative(d) , d^2)
end

In [ ]:
derivative(r1)

# Adding two rational types

In [ ]:
import Base: +

In [ ]:
function +(rf1::RationalFunction, rf2::RationalFunction)
    # a/b + c/d
    a, b = rf1.numerator, rf1.denominator
    c, d = rf2.numerator, rf2.denominator
    common = b*d
    return RationalFunction(a*d + c*b, common)
end

In [ ]:
r1+r2

In [ ]:
RationalFunction(1+x,x^2) + RationalFunction(3*one(PolynomialDense),x)

# Some sanity check

In [ ]:
prod_der_A = derivative(r1*r2)

In [ ]:
prod_der_B = r1*derivative(r2) + derivative(r1)*r2

Why are these different?

In [ ]:
function evaluate(r::RationalFunction, x::T) where T <: Number
    evaluate(r.numerator,x) // evaluate(r.denominator,x)
end

As you can see... they aren't different:

In [ ]:
evaluate(prod_der_A, 2), evaluate(prod_der_B, 2)

# Some operations that modify the polynomials

Say we wanted (for some obscure reason) to only have the polynomials with even absolute coefficients. That is, whenever there is a coefficient of the form $n$ for $nx^k$ then we must transform $n$ to be `abs(2*(n ÷ 2))`.

In [ ]:
clean(n::Int) = abs(2*(n÷2)) #\div + [TAB]

In [ ]:
[(n, clean(n)) for n=-5:5] |> println

In [ ]:
clean(t::Term) = Term(clean(t.coeff),t.degree)

In [ ]:
Term(1,3)

In [ ]:
cleaned = clean(Term(1,3))

In [ ]:
iszero(cleaned)

In [ ]:
function clean(p::PolynomialDense)
    p_out = PolynomialDense()
    terms = deepcopy(p).terms
    for t in terms
        clean_t = clean(t)
        !iszero(clean_t) && push!(p_out,clean(t))
    end
    return p_out
end

In [ ]:
PolynomialDense([Term(5,3),Term(2,2)])

In [ ]:
clean(PolynomialDense([Term(5,3),Term(2,2)]))

In [ ]:
using Random; Random.seed!(0)
p = rand(PolynomialDense) + 1x^50

In [ ]:
clean(p)

Say now we wanted to do this to the `RationalFunction` type:

In [ ]:
clean(r::RationalFunction) = RationalFunction(clean(r.numerator), clean(r.denominator))

In [ ]:
r1

In [ ]:
clean(r1)

## Abstract Types and Dispatch
Every object (number, variable, function, ...) in Julia has a concrete type, which can be found by querying `typeof`

In [ ]:
@show typeof(1.0)
@show typeof(1)
@show typeof(1//1)

These types are organised into collections known as Abstract types. We can recognise if a concrete type belongs to an Abstract type by using `<:` 

In [ ]:
@show Float64 <: Number #Float64 is a subtype of Number
@show Int <: Integer <: Number #Int is a shorthand (alias) for Int64, Int64 is a subtype of Integer which is a subtype of Number
@show Float64 <: Integer #Float64 is not a subtype of Integer

We can create our own concrete and abstract types. Consider a project involving points in 2D, we have two types of points, some are Integer valued, others are decimals. We can create two concrete types to store them. 

In [ ]:
struct PointFloat_attempt1
    x::Float64
    y::Float64
end

struct PointInt_attempt1
    x::Int
    y::Int
end

FloatPoints = [PointFloat_attempt1(3rand(), 3rand()) for _ in 1:10] #10 random floating points in [0,3]×[0,3]
IntPoints = [PointInt_attempt1(i,j) for i in 1:3 for j in 1:3] #9 integer points at x=(1,2,3), y=(1,2,3)

We can then define methods that accept these new types as arguments.

In [ ]:
function norm(point::PointFloat_attempt1)
    return sqrt(point.x^2 + point.y^2)
end

function norm(point::PointInt_attempt1)
    return sqrt(point.x^2 + point.y^2)
end

@show norm.(FloatPoints)
@show norm.(IntPoints)

Notice the actual code in the two methods is identical, we just needed to create two to handle both types. We could have instead used a shorthand to define both simultaneously. 

In [ ]:
function norm2(point::T) where {T <: Union{PointFloat_attempt1, PointInt_attempt1}} #the type T can be either PointFloat_attempt1 or PointInt_attempt1
    return sqrt(point.x^2 + point.y^2)
end

@show norm2.(FloatPoints)
@show norm2.(IntPoints)

# Making out own abstract types
We can simplify this by introducing an abstract type  

In [ ]:

abstract type Point2 end

Note we get an error if we try to redefine the existing types, either by declaring them as a subtype of trying to change the fields, which you can see by uncommenting the below. To make these changes we'd have to reboot the kernel before running the new definitions. We'll instead use a new name. 

In [ ]:

# struct PointFloat_attempt1 <: Point2 #error because we're changing the type structure
#     x::Float64
#     y::Float64
# end

# struct PointFloat_attempt1  
#     x::Float64
#     y::Float64
#     z::Float64 #error because we're changing the fields
# end

In [ ]:
struct PointFloat_attempt2 <: Point2 #the new concrete type PointFloat_attempt2 is declared as a subtype of the abstract type Point2
    x::Float64
    y::Float64
end

struct PointInt_attempt2 <: Point2
    x::Int
    y::Int
end

FloatPoints2 = [PointFloat_attempt2(3rand(), 3rand()) for _ in 1:10] #10 random floating points in [0,3]×[0,3]
IntPoints2 = [PointInt_attempt2(i,j) for i in 1:3 for j in 1:3] #9 integer points at x=(1,2,3), y=(1,2,3)

In [ ]:
norm(point::Point2) = sqrt(point.x^2 + point.y^2)

@show norm.(FloatPoints2)
@show norm.(IntPoints2);

If we want we can create a function specifically for a concrete type. This will be important in the project because `PolynomialDense` and `PolynomialSparse` will have different structures, and many functions will need to be specific to type. 

In [ ]:
function norm(point::PointInt_attempt2)
    println("Finding norm of an integer point")
    return sqrt(point.x^2 + point.y^2)
end

norm(IntPoints2[1])


When a function is called, it will dispatch to the most specific type it can find. Right now, `PointInt_attempt2` has 2 methods defined - one on its concrete type, and another on the abstract type `Point2` - the one on the concrete type is more specific (`PointInt_attempt2 <: Point2`), so will be used. The other concrete type `PointFloat_attempt2` still uses the norm defined on the abstract type, and therefore doesn't print any text. 

In [ ]:
norm(FloatPoints2[1])

We can see all the methods associated with a function name by using the `methods` function below, for any function that comes with Julia (+,-,*,...) or any packages being used you can expect very many methods

In [ ]:
methods(norm)

We can even reference the type of our arguments in the body of the function - here we use it to determine the colour when plotting the points. 

In [ ]:
using Plots
## This function inputs a vector containing elements of type T, where T is a subtype of Point2. This means T will be either PointInt_attempt2 or PointFloat_attempt2. If they are integer points they're plotted in blue, otherwise they are plotted in orange. 
function plotpoints(points::Vector{T}; plt=plot()) where {T <: Point2}
    plot!(plt, xlabel="x", ylabel="y", legend=:none)
    scatter!(
        [p.x for p in points], #unpack all x values from points
        [p.y for p in points], #unpack all y values from points
        color = (T==PointInt_attempt2) ? :lightblue : :orange  #check the type T, which is used to determine the colour
    )
end
plt = plotpoints(FloatPoints2)
plt = plotpoints(IntPoints2; plt=plt)

This feature is used in the project - for example consider this function from `src/polynomial_definitions/polynomial.jl` - it accepts an argument of type `P`, which must be a subtype of `Polynomial`. The first line of the function then creates an empty polynomial of type `P`. Currently the only subtype of `Polynomial` is `PolynomialDense`, but you'll create another in Task 3. 

In [ ]:
"""
Create a new polynomial which is the derivative of the polynomial.
"""
function derivative(p::P)::P where {P <: Polynomial} 
    der_p = P()
    for term in p
        der_term = derivative(term)
        !iszero(der_term) && push!(der_p, der_term)
    end
    return trim!(der_p)
end

We can combine these syntaxes to quickly generate many concrete types simultaneously, these are named [parametric types](https://docs.julialang.org/en/v1/manual/types/#Parametric-Types).

In [ ]:
struct Point_attempt3{T <: Real} <: Point2 #With this signature we're creating a new concrete type Point_attempt3{T} for every subtype of Real. All the new concrete types will be subtypes of Point2
    x::T
    y::T
end



FloatPoints3 = [Point_attempt3{Float64}(3rand(), 3rand()) for _ in 1:10] #10 random floating points in [0,3]×[0,3]
IntPoints3 = [Point_attempt3{Int}(i,j) for i in 1:3 for j in 1:3] #9 integer points at x=(1,2,3), y=(1,2,3)

In [ ]:
Point_attempt3{Int} <: Point_attempt3

These are declared as subtypes of the abstract type `Point2`, so any method defined on `Point2` will already work. 

In [ ]:
@show norm.(FloatPoints3)
@show norm.(IntPoints3)

plt = plotpoints(FloatPoints3)
plotpoints(IntPoints3; plt=plt)

Now our integers are orange rather than blue (why?), we can fix this by defining a new method on `Point_attempt3{Int}` alone.

In [ ]:
function plotpoints(points::Point_attempt3{Int}; plt=plot())
    plot!(plt, xlabel="x", ylabel="y", legend=:none)
    scatter!(
        [p.x for p in points], #unpack all x values from points
        [p.y for p in points], #unpack all y values from points
        color = :lightblue 
    )
end

plt = plotpoints(FloatPoints3)
plotpoints(IntPoints3; plt=plt) 

This still hasn't worked, where's the bug in the code cell above?

In [ ]:
methods(plotpoints)

There are now two (or more, depending on your changes) methods named `plotpoints`, which one is used when we call `plotpoints(FloatPoints3)`? How about when we call `plotpoints(IntPoints3; plt=plt)`?

Parametric typing like this is used extensively in `src\term.jl`, and you're asked to implement it yourself in task 2. 

## The Max Heap
The project includes a Max heap in `src/utils/heap.jl`, which you asked to use when implementing `PolynomialSparse` in Task 3. In the rest of this document we'll summarise how to generate and query these heaps. There are more details in the documentation written withen `heap.jl`. 

In [ ]:
methods(Heap)

We have two ways to generate a heap, the first inputs a type and generates an empty heap of that type, the second generates a heap based on an inputted vector. 

In [ ]:
@show Heap(Float64) #empty heap with Floats
@show Heap([5,19,21,1,7]) #heap with integers

The function `pop!` removes the largest element from the heap and returns it. Note that when displayed the heap may look unsorted, but `pop!` finds the largest value. 

In [ ]:
@show h1 = Heap([1,5,3,7,2])
@show pop!(h1)
@show h1 
@show pop!(h1)
@show h1

We can also add new elements to the heap using `push!`.

In [ ]:
h2 = Heap([11,2,31,9])
push!(h2, 31)
@show h2

In [ ]:
@show pop!(h2)
@show pop!(h2)
h2

We may also use `peek` to return the largest value without modifying the heap.  

In [ ]:
@show h2
@show peek(h2)
@show h2

We may also apply an operation to an entire heap using `map_heap` (or `map_heap!` to do so in-place), this may be useful when multiplying a sparse polynomial.

In [ ]:
h1 = Heap([1,5,3,7,2])
h2 = map_heap(h1, x->x^2) #square all elements in h1

In [ ]:
peek(h2)

This assumes that the function $f$ being mapped is order-preserving, meaning that $x>y$ implies $f(x)>f(y)$. Let's see what happens when our function isn't order-preserving. 

In [ ]:
f(x) = -x
h1 = Heap([1,5,3,7,2])
h2 = map_heap(h1, f) #negate all elements in h1

In [ ]:
peek(h2)

The order has now been broken, so now `pop!` and `peek` behave unpredictably, returning the smallest value, rather than the largest. 

We may also use `popall` to find the entire (sorted) vector associated with the heap (`popall!` to do so destructively)

In [ ]:
h1 = Heap([1,5,3,7,2])
h2 = map_heap(h1, x->x^2) #square all elements in h1
@show popall(h2)
@show isempty(h2)
@show h2
println() #print an empty line

h3 = Heap([1,5,3,7,2])
h4 = map_heap(h1, x->x^2) #square all elements in h1
@show popall!(h4)
@show isempty(h4)
@show h4;

Note the `isempty` function used above which, predictably, queries whether the heap is empty. Looking into `heap.jl` we can see that this uses our last convenience function, `length`, which measures the number of elements stored in a heap. 

In [ ]:
h1 = Heap(rand(10))
@show length(h1)
pop!(h1)
@show length(h1);